# Chorus and topic labelling

# Merging both dataframes, features from songs and features from lyrics

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('Limpieza.py','wb').write(src)
import Funciones

Saving Limpieza.py to Limpieza (4).py


In [ ]:
import pandas as pd
import numpy as np
from Funciones import limpieza
from Funciones import limpieza_chorus
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_songs_resampled_7637.csv')

In [ ]:
df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'playlist_name', 'playlist_id',
       'playlist_genre', 'playlist_subgenre', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'release_year', 'idioma',
       'lyrics', 'artist_popularity', 'track_album_release_month'],
      dtype='object')

In [ ]:
df_lyrics = pd.read_csv('/content/drive/MyDrive/df_lyrics.csv')

In [ ]:
df_lyrics['track_id'] = df.track_id

In [ ]:
df_lyrics.columns

Index(['clean_lyrics', 'len_words', 'riq_lex', 'most1', 'most1_freq', 'most2',
       'most2_freq', 'most3', 'most3_freq', 'less1', 'less1_freq', 'less2',
       'less2_freq', 'less3', 'less3_freq', 'len_ws', 'riq_lex_ws',
       'MostFreqPercentage', 'explicit', 'non_explicit', 'has_intro',
       'has_outro', 'chorus_count', 'prechorus_count', 'verse_count',
       'coef_chorus/len', 'verse_chorus/len', 'track_id'],
      dtype='object')

In [ ]:
# We merge on track_id
df_final = df.merge(df_lyrics, on='track_id', how='inner')

In [ ]:
df_final.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'playlist_name', 'playlist_id',
       'playlist_genre', 'playlist_subgenre', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'release_year', 'idioma',
       'lyrics', 'artist_popularity', 'track_album_release_month',
       'clean_lyrics', 'len_words', 'riq_lex', 'most1', 'most1_freq', 'most2',
       'most2_freq', 'most3', 'most3_freq', 'less1', 'less1_freq', 'less2',
       'less2_freq', 'less3', 'less3_freq', 'len_ws', 'riq_lex_ws',
       'MostFreqPercentage', 'explicit', 'non_explicit', 'has_intro',
       'has_outro', 'chorus_count', 'prechorus_count', 'verse_count',
       'coef_chorus/len', 'verse_chorus/len'],
      dtype='object')

# We create a feature for the chorus text

In [ ]:
import re

estribillos = []

for cancion in df_final.lyrics.values:
    text = cancion

    try:
        start_idx_chorus = text.find('Chorus')
        end_idx = text[start_idx_chorus+1:].find('[')
        end_idx += start_idx_chorus
        if end_idx > start_idx_chorus:
            estribillos.append(text[start_idx_chorus:end_idx])
        else:
            estribillos.append(cancion)

    except:
        estribillos.append(cancion)

In [ ]:
df_final['estribillos'] = estribillos

In [ ]:
df_final['clean_estribillos'] = df_final.estribillos.apply(limpieza)

In [ ]:
df_final['clean_estribillos'] = df_final['clean_estribillos'].astype('string')

In [ ]:
df_final['clean_estribillos'] = df_final['clean_estribillos'].apply(limpieza_chorus)

In [ ]:
df_final['len_estribillo_clean'] = df_final['clean_estribillos'].apply(lambda x: len(x))

In [ ]:
df_final['len_estribillo_clean'].describe()

count    7637.000000
mean      399.313081
std       633.370081
min         0.000000
25%       102.000000
50%       162.000000
75%       316.000000
max      5902.000000
Name: len_estribillo_clean, dtype: float64

In [ ]:
#If the song doesn't have a chorus, we will keep only 300 characters that is the mean of characters for all the instances
df_final.loc[df_final['chorus_count']==0,'clean_estribillos'] = df_final.loc[df_final['chorus_count']==0]['clean_estribillos'].apply(lambda x: x[:300])

In [ ]:
#We update the len of the chorus
df_final['len_estribillo_clean'] = df_final['clean_estribillos'].apply(lambda x: len(x))

In [ ]:
#Now our max number of characters is 2000, that belong to an entire chorus
df_final['len_estribillo_clean'].describe()

count    7637.000000
mean      187.405919
std       111.259830
min         0.000000
25%       102.000000
50%       162.000000
75%       300.000000
max      2023.000000
Name: len_estribillo_clean, dtype: float64

In [ ]:
df_final.to_csv('/content/drive/MyDrive/df_final.csv', index=False)

# LDA TOPIC MODELING - executed in collab T4 GPU

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/df_final.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7637 entries, 0 to 7636
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   track_id                   7637 non-null   object 
 1   track_name                 7637 non-null   object 
 2   track_artist               7637 non-null   object 
 3   track_popularity           7637 non-null   int64  
 4   track_album_id             7637 non-null   object 
 5   track_album_name           7637 non-null   object 
 6   playlist_name              7637 non-null   object 
 7   playlist_id                7637 non-null   object 
 8   playlist_genre             7637 non-null   object 
 9   playlist_subgenre          7637 non-null   object 
 10  danceability               7637 non-null   float64
 11  energy                     7637 non-null   float64
 12  key                        7637 non-null   int64  
 13  loudness                   7637 non-null   float

In [ ]:
#We want to know the type of the data in clean_estribillos, this should be a string but in some cases the cleaning function for chorus does not completely work because some lyrics obtained present different patterns
df['type'] = df['clean_estribillos'].apply(lambda x: type(x))
df['type'].value_counts()

<class 'str'>      7631
<class 'float'>       6
Name: type, dtype: int64

In [ ]:
# As we see, 6 instances have null values. Being 6 only, we decided to extract the chorus manually, this would be more simple seing the different cases
df['type'] = df['type'].apply(lambda x: str(x))
df['type'] = df['type'].apply(lambda x: x.strip('<'))
df['type'] = df['type'].apply(lambda x: x.strip('>'))

In [ ]:
df[df['type']=="class 'float'"][['lyrics', 'clean_lyrics', 'estribillos', 'clean_estribillos']]

,lyrics,clean_lyrics,estribillos,clean_estribillos
498,"[""[Instrumental Intro][Verse 1]Windin' your wa...",windin your way down baker street light your h...,Chorus,NaN
3031,"[""[Verse 1]Lately I've been thinking of one-wa...",lately ive been thinking one way ride maybe ta...,Chorus,NaN
5524,"[""[Verse 1]I’m reading through the Sunday pape...",reading through the sunday paper and all bad ...,Chorus,NaN
6517,"[""[Snoop Doggy Dogg Talking]No LimitYeah south...",limit yeah southern hospitality seizin you ni...,Chorus,NaN
6530,"[""[Whistling][Verse 1]I wanted love, I needed ...",wanted love needed love most all most all som...,Chorus,NaN
7471,"[""[Intro]Hmmm hmmm, hmmm, hmmm[Chorus] [?]You'...",hmmm hmmm hmmm hmmm youre the only thought tha...,Chorus],NaN


In [ ]:
df[df['type']=="class 'float'"].lyrics.values

array(['["[Instrumental Intro][Verse 1]Windin\' your way down on Baker StreetLight in your head and dead on your feetWell another crazy day, you\'ll drink the night awayAnd forget about everythingThis city desert makes you feel so coldIt\'s got so many people but it\'s got no soulAnd it\'s taking you so long to find out you were wrongWhen you thought it held everythingYou used to think that it was so easyYou used to say that it was so easyBut you\'re tryin\', you\'re tryin\' nowAnother year and then you\'d be happyJust one more year and then you\'d be happyBut you\'re cryin\', you\'re cryin\' now[Instrumental Chorus][Verse 2]Way down the street there\'s a light in his placeHe opens the door he\'s got that look on his faceAnd he asks you where you\'ve beenYou tell him who you\'ve seen and you talk about anythingHe\'s got this dream about buyin\' some landHe\'s gonna give up the booze and the one night standsAnd then he\'ll settle down, in some quiet little townAnd forget about everythin

In [ ]:
df.loc[498,'estribillos'] = 'It\'s got so many people but it\'s got no soulAnd it\'s taking you so long to find out you were wrongWhen you thought it held everythingYou used to think that it was so easyYou used to say that it was so easyBut you\'re tryin\', you\'re tryin\' nowAnother year and then you\'d be happyJust one more year and then you\'d be happyBut you\'re cryin\', you\'re cryin\' now'
df.loc[3031,'estribillos'] = 'Lately I\'ve been thinking of one-way flightMaybe to Hawaii or FranceI don\'t really know when I\'m coming homeBut I know, that I\'ll be back'
df.loc[5524,'estribillos'] = "So I’ll be singing (I’ll be singing)I’ll be singing (I’ll be singing)I’ll be singing in the rain (Singing in the rain)So I’ll be singing (I’ll be singing)I’ll be singing (I’ll be singing)I’ll be singing in the rain (Singing in the rain)I’ve got a song in my heart and I’m bulletproofThere’s nothing in the world that’s gonna kill this moodNo matter what life wants to throw my wayI’ll be singing, I’ll be singing in the rain"
df.loc[6517,'estribillos'] = 'All my real gangstaz throw yo hood upThrow yo hood upThrow yo hood up[Silkk:]    Now to all my real soldiers throw yo hood upThrow yo hood upThrow yo hood up[Snoop]Fancy cars, diamond ringsBad ass bitches, lots of cocaineSmokin on the best budGivin up that West loveGot them bitches shakin they ass, all in the club'
df.loc[6530,'estribillos'] = 'I wanted love, I needed loveMost of all, most of allSomeone said true love was deadAnd I\'m bound to fall, bound to fall for youOh, what can I do? Yeah Take my badge but my heart remainsLovin\' you, baby childTighten up on your reinsYou are runnin\' wild, runnin\' wild, it\'s true'
df.loc[7471,'estribillos'] = 'You\'re the only thought that\'s running through my headYou\'re the only monster hiding in my bedYou\'re the only reason I can feel so sadFeel so, so sad so sad soYou\'re the only thought that\'s running through my headYou\'re the only monster hiding in my bedYou\'re the only reason I can feel so sadFeel so, so sad so sad so'

In [ ]:
#Since the text we just added is not clean, we will be deleting the 'clean_estribillos' feature and creating it again

df.drop(columns='clean_estribillos', inplace=True)
df['clean_estribillos'] = df.estribillos.apply(limpieza)
df['clean_estribillos'] = df.clean_estribillos.apply(limpieza_chorus)

In [ ]:
#We do an LDA within all the text for obtaining topic labels

from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string

labels = []
# Preprocess text data (tokenization, removing stop words, etc.)
def preprocess_text(text):
    return preprocess_string(text)

# Preprocess your 'estribillos_clean' text column
processed_texts = [preprocess_text(text) for text in df['clean_estribillos']]

# Create a dictionary and corpus for LDA
dictionary = Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA model
num_topics = 5  # Adjust the number of topics as needed
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Get topics and top words for each topic
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)
    labels.append(topic)

(0, '0.049*"dont" + 0.042*"know" + 0.022*"let" + 0.022*"want" + 0.019*"wanna" + 0.017*"come" + 0.015*"time" + 0.013*"got" + 0.013*"your" + 0.013*"tell"')
(1, '0.033*"love" + 0.016*"feel" + 0.012*"awai" + 0.011*"heart" + 0.010*"like" + 0.009*"time" + 0.009*"life" + 0.009*"world" + 0.009*"ill" + 0.008*"run"')
(2, '0.078*"ooh" + 0.033*"ayi" + 0.020*"yeah" + 0.020*"babi" + 0.016*"whoa" + 0.013*"woah" + 0.013*"woo" + 0.009*"hot" + 0.008*"boi" + 0.007*"got"')
(3, '0.034*"yeah" + 0.028*"man" + 0.026*"like" + 0.020*"gonna" + 0.017*"hei" + 0.014*"girl" + 0.014*"night" + 0.013*"danc" + 0.012*"got" + 0.010*"your"')
(4, '0.028*"nigga" + 0.021*"like" + 0.017*"got" + 0.016*"fuck" + 0.015*"bitch" + 0.014*"shit" + 0.012*"aint" + 0.010*"yeah" + 0.008*"monei" + 0.007*"caus"')


# Obtaining topic labels for every instance

In [ ]:
# Seing the previous results, we decide that it would be interesting to label each instance with topic tags
# Getting 3 labels for instance

from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string

# Preprocess text data (tokenization, removing stop words, etc.)
def preprocess_text(text):
    return preprocess_string(text)

# Preprocess your 'clean_estribillos' text column
processed_texts = [preprocess_text(text) for text in df['clean_estribillos']]

# Create a dictionary and corpus for LDA
dictionary = Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Train LDA model
num_topics = 5  # Adjust the number of topics as needed
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Function to get top words for each topic
def get_top_words_for_topic(topic_id, num_words=3):
    top_words = lda_model.show_topic(topic_id, topn=num_words)
    return [word for word, _ in top_words]

# Assign a topic with 3 words to each instance in the DataFrame
for i, text in enumerate(df['clean_estribillos']):
    bow = dictionary.doc2bow(preprocess_text(text))
    topic_distribution = lda_model.get_document_topics(bow)
    top_topic = max(topic_distribution, key=lambda x: x[1])[0]
    top_words_for_instance = get_top_words_for_topic(top_topic)
    df.loc[i, 'assigned_topic_words'] = ', '.join(top_words_for_instance)

# Display the DataFrame with assigned topic words
print(df[['clean_estribillos', 'assigned_topic_words']])

                                      clean_estribillos assigned_topic_words
0      lac lil wayne baby ima hard stunin nigga like...     nigga, got, like
1      you you only one you number one you one one w...     love, dont, know
2      ten bands twenty bands what shell anything fo...     nigga, got, like
3      look eyes see you everytime found inside perf...     love, dont, know
4      future meek mill perrier jouët tap water this...     nigga, got, like
...                                                 ...                  ...
7632  ive been thinkin bout all the times you told y...     love, dont, know
7633   why not try everything why stop reach for any...     love, dont, know
7634  let see hold hol let see hol let see let see l...     nigga, got, like
7635   now heres little something that needs heard u...     nigga, got, like
7636  every single candlelight sets blaze they burn ...   come, gonna, heart

[7637 rows x 2 columns]


In [ ]:
# We see that there are only 5 unique values in 'assigned_topic_words'
df['assigned_topic_words'].unique()

array(['nigga, got, like', 'love, dont, know', 'like, man, hei',
       'come, gonna, heart', 'boom, stand, tear'], dtype=object)

In [ ]:
# Since there are only 5 different options, we decide to create a column for every topic word that we will treat as categorical features

df['assigned_topic_words'] = df['assigned_topic_words'].apply(lambda x: x.split())
df['topic1'] = df['assigned_topic_words'].apply(lambda x: x[0])
df['topic2'] = df['assigned_topic_words'].apply(lambda x: x[1])
df['topic3'] = df['assigned_topic_words'].apply(lambda x: x[2])

# We delete columns we will not be using
df.drop(columns=['estribillos', 'clean_estribillos', 'type', 'assigned_topic_words'], inplace=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/df_final.csv', index=False)